In [1]:
import sys
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("Spark Session") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS silver")

DataFrame[]

In [47]:
df = spark.sql("""
SELECT 
DISTINCT
Identificador, 
cast(to_date(from_unixtime(unix_timestamp(Data, 'dd/MM/yyyy'))) as date) AS Data,
cast(to_date(from_unixtime(unix_timestamp(Data, 'dd/MM/yyyy'))) as date) AS Data_Pgto,
Descricao,
CASE WHEN Valor < 0 THEN 'PAGO' ELSE 'RECEBIDO' END AS Tipo,
Valor
FROM bronze.nubank_extrato
""")

In [48]:
df.coalesce(1).write.mode("overwrite").format("parquet").saveAsTable(f"silver.nubank_extrato")

In [49]:
df = spark.sql("""
SELECT
DISTINCT
date AS Data,
CAST(SUBSTRING(arquivo_origem,8,10) AS date) AS Data_Pgto,
title AS Descricao,
'PAGO' AS Tipo,
(amount * -1) AS Valor
FROM bronze.nubank_fatura
""")

In [50]:
df.coalesce(1).write.mode("overwrite").format("parquet").saveAsTable(f"silver.nubank_fatura")